In [1]:
import numpy as np
import pandas as pd
from ideas import intersection_over_union
from ideas import areas
user_columns = ['iou', 'userId']

In [6]:
t_d = pd.read_csv('train_data.csv')
t_a = pd.read_csv('train_answers.csv')
t_e = pd.read_csv('test_data.csv')
t_e_a = pd.read_csv('test_data_fgd.csv')
ids = list(set(t_d['userId']).union(set(t_e['userId'])))
t_w_a = t_d.merge(t_a, on=["itemId"])
t_e_w_a = t_e.merge(t_e_a, on=["itemId"])

In [8]:
user_rating = pd.DataFrame(np.zeros((len(ids), 2), dtype=int), 
                           index=ids, columns=['iou','userId'])
t_w_a['iou'] = t_w_a[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
t_w_a = t_w_a.groupby("userId")[user_columns].mean()

t_e_w_a['iou'] = t_e_w_a[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_fgd',\
      'Ymin_fgd', 'Xmax_fgd','Ymax_fgd']].apply(intersection_over_union, axis=1)
t_e_w_a = t_e_w_a.groupby("userId")[user_columns].mean()

In [12]:
for i in ids:
    if i in t_w_a.index:
        user_rating.loc[i] = t_w_a.loc[i]
    else:
        user_rating.loc[i] = t_e_w_a.loc[i]

In [13]:
user_rating.to_csv('users_rating.csv', index=False)